In [1]:
# Using Python 3.12.1 (local env: HeatNSalt)

# Merge DNA data with Imaging / BioDiscover data

import pandas as pd

setup_data = pd.read_csv('outputs/setup_data_complete.csv')
OTUs = pd.read_excel('outputs/EPT_OTU_reads.xlsx', sheet_name='Top Hits')
weights = pd.read_excel('raw/specimen_weights.xlsx')

weights['Specimen Weight'] = weights['Tube Weight with dried Specimen'] - weights['Empty Tube Weight']
weights.pop('Unnamed: 0')
OTUs_W = OTUs[OTUs['Sample'].str.startswith('W')]
OTUs_W['Repeated Sample'] = OTUs_W['Sample'].str[2:]
merged = pd.merge(weights, OTUs.add_suffix('_DNA'), how='left', left_on='Sample Name/Number', right_on='Sample_DNA')
merged_w = pd.merge(merged, OTUs_W.add_suffix('_DNA_repeat'), how='left', left_on='Sample Name/Number', right_on='Repeated Sample_DNA_repeat')

merged_w.to_excel('outputs/EPT_data_merged.xlsx', index=False) # includes both, repeated and not repeated samples

image_data = pd.read_csv('outputs/image_data_complete.csv').dropna(axis=1)
image_data.insert(0, 'Specimen ID', image_data['Sample Name/Number'].str.split('_').str[:-1].str.join('_')) # create new column with specimen ID from Sample Name/Number

image_data_merged = pd.merge(image_data, merged_w, how='left', left_on='Specimen ID', right_on='Sample Name/Number').dropna(axis=1, how='all')
image_data_merged.drop(['Sample Name/Number_y', 'Sample_DNA', 'Sample_DNA_repeat', 'Repeated Sample_DNA_repeat'], axis=1, inplace=True)
image_data_merged = image_data_merged.rename(columns={'Sample Name/Number_x': 'Sample Name/Number', 'Order': 'Order_Morpho'})

image_data_merged_filtered = image_data_merged[(image_data_merged['Order_Morpho'] == image_data_merged['Order_DNA']) | (image_data_merged['Order_Morpho'] == image_data_merged['Order_DNA_repeat'])].reset_index(drop=True)

image_data_merged.to_csv('outputs/image_data_merged.csv', index=False)
image_data_merged_filtered.to_csv('outputs/image_data_with_DNA.csv', index=False)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/var/folders/b0/68bb2dkj42g1jr38kdd8m6pc0000gp/T/ipykernel_32360/1902755807.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  OTUs_W['Repeated Sample'] = OTUs_W['Sample'].str[2:]
